In [36]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [33]:
batchsize=8

In [40]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\Python37\Projects\iNeuron Intership Projects\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\Python37\Projects\iNeuron Intership Projects\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64263 images belonging to 2 classes.
Found 16065 images belonging to 2 classes.


In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\Python37\Projects\iNeuron Intership Projects\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4570 images belonging to 2 classes.


In [27]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_752 (Conv2D)             (None, 39, 39, 32)   864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_752 (BatchN (None, 39, 39, 32)   96          conv2d_752[0][0]                 
__________________________________________________________________________________________________
activation_752 (Activation)     (None, 39, 39, 32)   0           batch_normalization_752[0][0]    
____________________________________________________________________________________________

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [42]:
checkpoint = ModelCheckpoint(r'D:\Python37\Projects\iNeuron Intership Projects\CV_Driver_Drowsiness_Detection\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5
8032/8032 [==============================] - ETA: 0s - loss: 0.1712 - accuracy: 0.9344
Epoch 00001: val_loss improved from inf to 0.20740, saving model to D:\Python37\Projects\iNeuron Intership Projects\CV_Driver_Drowsiness_Detection\models\model.h5
8032/8032 [==============================] - 405s 50ms/step - loss: 0.1712 - accuracy: 0.9344 - val_loss: 0.2074 - val_accuracy: 0.9106 - lr: 0.0010
Epoch 2/5
 237/8032 [..............................] - ETA: 5:37 - loss: 0.1519 - accuracy: 0.9388 ETA: 5:35 - loss: 0.1487 - 

KeyboardInterrupt: 

In [ ]:
# Model Evaluation

In [44]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

Instructions for updating:
Please use Model.evaluate, which supports generators.


KeyboardInterrupt: 

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)